In [85]:
import itertools
import pickle
import re
import pandas
import numpy
import sys
import math

In [86]:
def chunks(l, n):
    # For item i in a range that is a length of l,
    for i in range(0, len(l), n):
        # Create an index range for l of n items:
        yield l[i:i+n]

In [90]:
alphabet = ["a","c", "g", "t", "[ag]", "[ct]", "."]
n = 8
allMotifs = list(itertools.product(alphabet,repeat=n)) #generates allmotifs of length n
num_jobs = 100000
job_number = 10
# job_number = sys.argv[1]
parts = list(chunks(allMotifs, round(len(allMotifs)/num_jobs)))
print(len(parts))
if(job_number > len(parts)):
    quit()
motifs_for_this_job = parts[job_number]
len(motifs_for_this_job)
# len(parts)

99394


58

In [91]:
protein_to_sequence = pickle.load( open( "files/intermediates/proteins_to_sequences.pkl", "rb" ) )
protein_to_shuffled_sequence = pickle.load( open( "files/intermediates/proteins_to_shuffled_sequences.pkl", "rb" ) )
distanceMatrix_df = pandas.read_pickle(open( "files/intermediates/distanceMatrix.pkl", "rb" ))

In [33]:
motifs_to_proteins = {}
motifs_to_proteins_shuffled = {}
for motif in motifs_for_this_job:
    motif = "".join(motif) #turning motif array into string
    motifs_to_proteins[motif] = []
    motifs_to_proteins_shuffled[motif] = []
    print(motif)
    for protein in protein_to_sequence:
#         if(motif in protein_to_sequence[protein]):
        if(re.search(motif, protein_to_sequence[protein]) != None):
            motifs_to_proteins[motif].append(protein)
            
    for protein in protein_to_shuffled_sequence:
#         if(motif in protein_to_sequence[protein]):
        if(re.search(motif, protein_to_shuffled_sequence[protein]) != None):
            motifs_to_proteins_shuffled[motif].append(protein)

aaaaacc[ag]
aaaaacc[ct]
aaaaacc.
aaaaacga
aaaaacgc
aaaaacgg


In [92]:
%%time
motif_to_size = {}
motifs_to_average_scores = {}
motifs_to_tt_average_scores = {}

i = 0
for motif in motifs_to_proteins:
    motif_to_size[motif] = len(motifs_to_proteins[motif])
#     making subgraph
    if( len(motifs_to_proteins[motif]) < 2 or  len(motifs_to_proteins[motif]) > 1000):
        continue
    proteins = motifs_to_proteins[motif]
    subgraph = [];
    for protein_i in proteins:
        row = []
        for protein_j in proteins:
            row.append(distanceMatrix_df[protein_i][protein_j])
        subgraph.append(row)
#     scoring
    avg_score = pandas.DataFrame(subgraph).mean().mean()
    ninety_p = numpy.percentile(subgraph, 90)
    for i, row in enumerate(subgraph):
        for j, elem in enumerate(row):
            if(elem < math.floor(ninety_p)):
                subgraph[i][j] = None
    tt_avg_score = pandas.DataFrame(subgraph).mean().mean()
    
    motifs_to_average_scores[motif] = avg_score
    motifs_to_tt_average_scores[motif] = tt_avg_score


CPU times: user 6.67 s, sys: 7.53 ms, total: 6.67 s
Wall time: 6.69 s


In [93]:
%%time
shuffled_motif_to_size = {}
shuffled_motifs_to_average_scores = {}
shuffled_motifs_to_tt_average_scores = {}

for motif in motifs_to_proteins_shuffled:
    shuffled_motif_to_size[motif] = len(motifs_to_proteins_shuffled[motif]) 
# preparing subgraph
    if( len(motifs_to_proteins_shuffled[motif]) < 2 or  len(motifs_to_proteins_shuffled[motif]) > 1000):
        continue
    proteins_shuffled = motifs_to_proteins_shuffled[motif]
    shuffled_subgraph = [];
    for protein_shuffled_i in proteins_shuffled:#shuffled subgraph
        row_shuffled = []
        for protein_shuffled_j in proteins_shuffled:
#             if(protein_shuffled_i != protein_shuffled_j):
            row_shuffled.append(distanceMatrix_df[protein_shuffled_i][protein_shuffled_j])
        shuffled_subgraph.append(row_shuffled)

    shuffled_avg_score = pandas.DataFrame(shuffled_subgraph).mean().mean()
    ninety_p_shuffled = numpy.percentile(shuffled_subgraph, 90)
    for i, row in enumerate(shuffled_subgraph):
        for j, elem in enumerate(row):
            if(elem < math.floor(ninety_p_shuffled)):
                shuffled_subgraph[i][j] = None
    tt_shuffled_avg_score = pandas.DataFrame(shuffled_subgraph).mean().mean()
    
    shuffled_motifs_to_average_scores[motif] = shuffled_avg_score
    shuffled_motifs_to_tt_average_scores[motif] = tt_shuffled_avg_score


CPU times: user 5.2 s, sys: 3.57 ms, total: 5.2 s
Wall time: 5.2 s


In [22]:
# %%time
# motifs_to_average_scores = {}
# motifs_to_tt_average_scores = {}


# for motif in motifs_to_subgraphs:
#     subgraph = motifs_to_subgraphs[motif]
#     avg_score = pandas.DataFrame(subgraph).mean().mean()
    
#     ninety_p = numpy.percentile(subgraph, 90)
#     for i, row in enumerate(subgraph):
#         for j, elem in enumerate(row):
#             if(elem < math.floor(ninety_p)):
#                 subgraph[i][j] = None
#     tt_avg_score = pandas.DataFrame(subgraph).mean().mean()
    
#     motifs_to_average_scores[motif] = avg_score
#     motifs_to_tt_average_scores[motif] = tt_avg_score

CPU times: user 517 ms, sys: 9.65 ms, total: 527 ms
Wall time: 527 ms


In [23]:
# %%time
# shuffled_motifs_to_average_scores = {}
# shuffled_motifs_to_tt_average_scores = {}

# for motif in motifs_to_shuffled_subgraphs:
#     shuffled_subgraph = motifs_to_shuffled_subgraphs[motif]
#     shuffled_avg_score = pandas.DataFrame(shuffled_subgraph).mean().mean()
    
#     ninety_p_shuffled = numpy.percentile(shuffled_subgraph, 90)
#     for i, row in enumerate(shuffled_subgraph):
#         for j, elem in enumerate(row):
#             if(elem < math.floor(ninety_p_shuffled)):
#                 shuffled_subgraph[i][j] = None
#     tt_shuffled_avg_score = pandas.DataFrame(shuffled_subgraph).mean().mean()
    
#     shuffled_motifs_to_average_scores[motif] = shuffled_avg_score
#     shuffled_motifs_to_tt_average_scores[motif] = tt_shuffled_avg_score

CPU times: user 406 ms, sys: 2.61 ms, total: 409 ms
Wall time: 408 ms


In [194]:
df = pandas.DataFrame({"motif_subgraph_size" : motif_to_size, "average_scores":motifs_to_average_scores, "average_tt_scores" : motifs_to_tt_average_scores,
                      "motif_shuffled_subgraph_size" : shuffled_motif_to_size, "shuffled_average_scores" : shuffled_motifs_to_average_scores, "shuffled_average_tt_scores" : shuffled_motifs_to_tt_average_scores})
df['average_scores']['aaaaacgg']
# motif_to_size

nan

In [182]:
motif_to_size

{'aaaaacc.': 616,
 'aaaaacc[ag]': 312,
 'aaaaacc[ct]': 371,
 'aaaaacga': 46,
 'aaaaacgc': 27}

In [181]:
del motifs_to_tt_average_scores['aaaaacgg']

In [186]:
df.to_pickle("files/numJobs:" + str(num_jobs) + "_jobNumber:" + str(job_number) + ".pkl")

In [232]:
len(allMotifs)

5764801

3min 0-50
30min 50-100 
60min 100-150
120   150-200
140.  200-250
200.  250-300 X(failed at 290)
270   300-350 X(failed at 334)
bump up to actual
372
478   350-400
546   400-450
676.  450-500
751.  500-550

In [158]:
t1 = 3
t2 = 30
t3 = 60
t4 = 120
t5 = 140
t6 = t5* 200/150 * 1.1
t7 = t6* 250/200 * 1.1
t8 = t7* 300/250 * 1.1
# t8 = t8 *1.2
t9 = t8* 350/300 * 1.1
t10 = t9* 400/350 
t11 = t10* 450/400  *1.1
t12 = t11* 500/450  
t13 = t12 * (550/500)**2
t8






372.6800000000001

In [160]:
t0 = 0.5 #ten
t2 = 0.5 * 25 #0-50
t2 = t1 * (100/50)**2 #50-100
t3 = t2 * (150/100)**2#100-150
t4 = t3 * (200/150)**2#150-200
t6 = t5* (250/200)**2#200-250
t7 = t6* (250/200)**2#250-300
t8 = t7* (300/250)**2#300-350
t9 = t8* (350/300)**2#350-400
t10 = t9* (400/350) **2#400-450
t11 = t10* (450/400)**2#450-500
t12 = t11* (500/450)**2 #500-550
t13 = t12 * (550/500)**2#550-600

t9



669.9218750000001

In [156]:
ten = 0.3
ti = ten*(5**2)
tsum = t1*50
print(0, 50 , ti)
for i in range(2,round(500/50)+1):
    start_range = (i-1)*50
    end_range = i*50
    ti = ti * (end_range/start_range)**2
    tsum+= ti*50
    print(start_range, end_range , ti)
print(tsum/60/24/365)

0 50 7.5
50 100 30.0
100 150 67.5
150 200 120.0
200 250 187.5
250 300 270.0
300 350 367.50000000000006
350 400 480.00000000000006
400 450 607.5000000000001
450 500 750.0000000000002
0.27444824961948244


In [157]:
150*1000/60/24/365

0.2853881278538813

In [45]:
324 *1.1 *400/350

503.6625000000001

In [46]:
503 *1.1 *500/450

614.7777777777779

In [47]:
614 *1.1 *550/500

742.9400000000002

In [84]:
parts[999]

[('.', '.', '.', '[ag]', '[ag]', '[ct]', 'g', '.'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', 't', 'a'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', 't', 'c'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', 't', 'g'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', 't', 't'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', 't', '[ag]'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', 't', '[ct]'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', 't', '.'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', '[ag]', 'a'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', '[ag]', 'c'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', '[ag]', 'g'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', '[ag]', 't'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', '[ag]', '[ag]'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', '[ag]', '[ct]'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', '[ag]', '.'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', '[ct]', 'a'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', '[ct]', 'c'),
 ('.', '.', '.', '[ag]', '[ag]', '[ct]', '[ct]', 'g'),
 ('.', '.', '.', '[ag]